In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer,OneHotEncoder,OrdinalEncoder
from category_encoders import MEstimateEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA


from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR

from tensorflow import keras 
import tensorflow as tf

from xgboost import XGBRegressor


2023-10-13 21:44:15.100945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-13 21:44:15.101048: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:

train_df = pd.read_csv('train.csv',index_col=0)
test_df = pd.read_csv('test.csv')


X_train = train_df.drop('SalePrice', axis=1)
y_train = train_df['SalePrice']

In [ ]:
X_train.describe()

In [ ]:
print(X_train.isnull().sum())

In [ ]:
numerical_cols=X_train.select_dtypes(exclude=['object'])
categorical_cols=X_train.select_dtypes(include=['object'])
cols_with_missing_values=[cols for cols in X_train.columns if X_train[cols].isnull().sum().any()]
print(cols_with_missing_values)


numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_imputer, numerical_cols.columns),
        ('cat', categorical_imputer, categorical_cols.columns)
    ])

X_train_imputed = pd.DataFrame(preprocessor.fit_transform(X_train))
print(X_train_imputed.isnull().sum().any())

In [ ]:
for col in categorical_cols.columns:
    num_unique = categorical_cols[col].nunique()
    print(f"Column {col} has {num_unique} unique values.")

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.figure(figsize=(15,15))
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    plt.show()

In [ ]:
mi_score=make_mi_scores(X_train_imputed,y_train)
plot_mi_scores(mi_score)